In [1]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
load_dotenv()
import os
import json
import fitz 
import os
import tempfile


In [2]:
client=genai.Client(api_key=os.getenv("gemini_api"))

In [3]:
file=client.files.upload(file=r"C:\\Users\\surya\\AppData\\Local\\Temp\\pdf_pages_melr2x2o\\page_1.pdf")

FileNotFoundError: C:\\Users\\surya\\AppData\\Local\\Temp\\pdf_pages_melr2x2o\\page_1.pdf is not a valid file path.

In [ ]:
system_prompt="""You are an advanced OCR (Optical Character Recognition) engine. Your task is to perform a full-fledged, page-wise transcription of the provided document with the highest possible accuracy and without any hallucinations.
Instructions:
Page-by-Page Processing: Process the document strictly on a page-by-page basis. Each page's extracted text should be clearly delineated and indexed by its corresponding page number.
Comprehensive Text Extraction: Extract all textual content from each page, including:
Main body text
Headers and footers
Text within tables
Captions for images and figures
Footnotes and endnotes
Text within graphical elements (if legible)
Structured Output Format: Present the output in a structured format, preferably JSON, with a clear hierarchy. Each page object should contain its full text content. For example:
code
JSON
{
  "page": [
    {
      "content": "[Full text content of page]"
    }
    ...
  ]
}
Preserve Original Formatting (as much as possible):
Maintain the original paragraph breaks.
Recognize and transcribe lists (numbered and bulleted) accurately.
For tabular data, attempt to preserve the row and column structure within the text output, using clear delimiters like " | " for cells and newlines for rows.
Highest Accuracy and No Hallucinations:
Transcribe the text exactly as it appears in the document.
Do not infer, guess, or add any information that is not explicitly present in the source document.
If a word or character is illegible or unclear, represent it with a specific placeholder (e.g., [ILLEGIBLE]). Do not attempt to guess the content.
Pay close attention to special characters, symbols, and diacritics, and transcribe them accurately.
Language and Character Set:
The document is primarily in English. Be prepared to handle any other languages or special character sets, e.g., mathematical equations, chemical formulas if they appear."""

In [75]:
test_prompt_2="""You are an advanced OCR (Optical Character Recognition) engine. Your task is to perform a full-fledged, page-wise transcription of the provided document with the highest possible accuracy and without any hallucinations.
Instructions
1. Page-by-Page Processing
Process the document strictly on a page-by-page basis. Each page's extracted text should be clearly delineated and indexed by its corresponding page number.
2. Comprehensive Text Extraction
Extract all textual content from each page, including but not limited to:
Main body text
Headers and footers
Text within tables
Captions for images and figures
Footnotes and endnotes
Text within graphical elements (if legible)
3. Structured Output Format
Present the output in a structured JSON format with a clear hierarchy. Each page object should contain its full text content.
Example JSON Structure:
code
JSON
{
  "page": [
    {
      "content": "[Full text content of page]"
    }
    ...
  ]
}
4. Preserve Original Formatting (as much as possible)
Formatting Element	Instruction
Paragraphs	Maintain the original paragraph breaks.
Lists	Recognize and transcribe lists (numbered and bulleted) accurately.
Tabular Data	Attempt to preserve the row and column structure within the text output, using clear delimiters like " | " for cells and newlines for rows.
5. Highest Accuracy and No Hallucinations
Transcribe the text exactly as it appears in the document.
Do not infer, guess, or add any information that is not explicitly present in the source document.
If a word or character is illegible or unclear, represent it with a specific placeholder (e.g., [ILLEGIBLE]). Do not attempt to guess the content.
Pay close attention to special characters, symbols, and diacritics, and transcribe them accurately.
6. Language and Character Set
The primary language of the document is English. Be prepared to handle any other languages or special character sets, such as mathematical equations or chemical formulas, if they appear."""

In [9]:
ocr_prompt="""You are an advanced OCR (Optical Character Recognition) engine. Your primary task is to transcribe the provided document into a single Markdown file. The goal is to ensure the rendered Markdown file visually imitates the original document's layout, structure, and formatting as perfectly as possible.
Core Instructions
Page-by-Page Processing: Transcribe the document sequentially, page by page. 



Comprehensive Text Extraction: You must extract all textual content from each page, including headers, footers, main body text, table content, captions, and footnotes.

No Hallucinations: Transcribe the text exactly as it appears. Do not infer, guess, or add information that is not explicitly present. If a word or character is illegible, represent it with the placeholder [ILLEGIBLE].


## 3. STRUCTURAL & FORMATTING RULES

You **must** use the following Markdown syntax to represent the corresponding document elements:

| Document Element          | Markdown Implementation                                                                                                                  |
| :------------------------ | :--------------------------------------------------------------------------------------------------------------------------------------- |
| **Headings**              | Use appropriate heading levels (`#`, `##`, `###`, etc.) to match the visual hierarchy of titles and subtitles.                               |
| **Paragraphs**            | Preserve original paragraph breaks with a blank line between them.                                                                       |
| **Bold & Italics**        | Use `**bold text**` for bolded text and `*italicized text*` for italics or underlined text.                                                 |
| **Lists**                 | Replicate numbered (`1.`, `2.`) and bulleted (`*`, `-`) lists exactly, including any indentation for nested items.                         |
| **Tables**                | Reconstruct tables using Markdown's native pipe syntax. **Do not use HTML (`<table>`, `<tr>`, `<td>`).**                                   |
| **Headers & Footers**     | Place header/footer text at the very top/bottom of each page's section, separated from the main content by a horizontal rule (`---`).       |
| **Image/Figure Placeholders** | Represent images or figures with a descriptive placeholder like `[IMAGE: A bar chart showing quarterly profits]`.                     |
| **Captions**              | Place captions directly below their corresponding placeholder and format them in italics (e.g., `*Figure 1: Quarterly Profit Growth.*`).      |
| **Footnotes**             | Use Markdown footnote syntax: place `[^1]` in the text and define the footnote `[^1]: Footnote text here.` at the bottom of the page section. |
| **Blockquotes**           | For indented blocks of text or quotes, use the blockquote symbol (`>`).                                                                  |
| **Code / Preformatted Text** | Use triple backticks (```) to enclose blocks of code or preformatted text.                                                                |

Response format:

JSON
{
  "page": [
    {
      "content": "[Full text content of page]"
    }
    ...
  ]
}

"""


In [76]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[file],
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0
    )
)

In [4]:
def parse_json(json_output: str):
    # Parsing out the markdown fencing
    lines = json_output.splitlines()
    for i, line in enumerate(lines):
        if line == "```json":
            json_output = "\n".join(lines[i+1:]) 
            json_output = json_output.split("```")[0]  
            break  
    return json_output
     

In [5]:
def split_pdf_to_pages(pdf_path: str) -> dict[int, str]:
    """
    Splits a PDF into individual one-page PDFs.
    
    Args:
        pdf_path (str): Path to the input PDF file.
        
    Returns:
        dict[int, str]: A dictionary mapping page_number -> single-page PDF file path.
    """
    # Creates a temporary directory to store the split pages
    output_dir = tempfile.mkdtemp(prefix="pdf_pages_")

    # Opens the original PDF
    doc = fitz.open(pdf_path)
    page_dict = {}

    # Iterates through each page
    for page_num in range(len(doc)):
        single_page_doc = fitz.open()         # create a new empty PDF
        single_page_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)

        # Construct the output file path
        output_path = os.path.join(output_dir, f"page_{page_num+1}.pdf")
        single_page_doc.save(output_path)
        single_page_doc.close()

        # Stores in dictionary (1-based page number)
        page_dict[page_num + 1] = output_path

    doc.close()
    return page_dict


In [6]:
file_test=r"C:\Users\surya\OneDrive\Desktop\Projects\DocumentAI\Test_docs\brochure.pdf"
page_dict=split_pdf_to_pages(file_test)

In [7]:
import re

In [8]:
all_page_data=[]

for page_number, page_path in page_dict.items():
    print("Parsing page:",page_number)
    file=client.files.upload(file=page_path)
    response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[file],
    config=types.GenerateContentConfig(
        system_instruction=ocr_prompt,
        temperature=0
    )
)
    data={"page_number":page_number,"ocr": None}
    # Parsing the model's response
    parsed_data = parse_json(response.text)
    if parsed_data:
        try:
            # If parse_json is dict 
            if isinstance(parsed_data, dict):
                json_data = parsed_data
            else:
                # If it is a string
                safe_str = re.sub(r'[\x00-\x1F\x7F]', ' ', parsed_data)
                json_data = json.loads(safe_str)

                # json_data = json.loads(parsed_data)

            data["ocr"] = json_data.get("page", [{}])[0].get("content")

        except Exception as e:
            print(f"Error parsing page {page_number}: {e}")

    all_page_data.append(data)

output_json_path = "ocr_output_new.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(all_page_data, f, indent=4, ensure_ascii=False)

print(f"OCR data saved to: {output_json_path}")

Parsing page: 1
Parsing page: 2
Parsing page: 3
Parsing page: 4
Parsing page: 5
Parsing page: 6
Parsing page: 7
Parsing page: 8
Parsing page: 9
Parsing page: 10
Parsing page: 11
OCR data saved to: ocr_output_new.json


In [82]:
print(parsed_data["page"][0]["content"])

RERA Registration No. UPRERAPRJ250823/04/2025
dated 30-04-2025 (website: www.up-rera.in)

GODREJ MAJESTY
SECTOR 12, GREATER NOIDA (W)

FOR YOUR EXCELLENCE

Godrej PROPERTIES

Godrej Properties Limited (CIN- L74120MH1985PLC035308) ("Company") is developing a residential group housing project Godrej Majesty
("Project"), situated at Plot No. GH-01/B,C,D,E.J and K, Sec 12, Gr. Noida, which is registered with Uttar Pradesh Real Estate Regulatory
Authority vide RERA Registration No. UPRERAPRJ250823/04/2025 dated 30/04/25 (website: www.up-rera.in). The Project is being developed,
pursuant to permit no. SM-10-Dec-2024:23309 dated 16/02/25 valid for 5 years granted by Gr. Noida Authority for the Project and any further
revisions and renewals in future. The terms of allotment/sale shall be subject to documents executed with the Company and approvals
(as renewed time to time).


In [12]:
model_2="gemini-2.5-flash-lite"

In [14]:
sys_prompt_2="""You are an advanced Document Layout Analysis engine. Your primary function is to interpret the structural layout of a document page.
You will be given the raw text content of a document, organized by page number (the output from a high-accuracy OCR process). Your task is to analyze this text in the context of a document's visual structure and generate a detailed, structured JSON output that identifies, chunks, and provides precise bounding boxes for every significant element on each page.
Instructions:
Input: Your input will be a JSON object containing the page-wise text content. It will follow this format:
code
JSON
{
  "document_title": "Example Document",
  "total_pages": 1,
  "pages": [
    {
      "page_number": 1,
      "content": "Document Title\n\nThis is the first paragraph of the document. It contains several sentences to illustrate a block of text.\n\nAnother Paragraph\nThis is the second paragraph. It follows the first one after a clear separation."
    }
  ]
}
Core Task: Identify and Chunk Elements: For each page, analyze its content to identify and segment distinct logical blocks. You must identify the following element types:
Headings: (e.g., heading_1, heading_2)
Paragraphs: (paragraph)
Tables: (table)
Lists: and their individual items (list, list_item)
Images: (image) - Note: You won't have the image data, but you should infer its location based on surrounding text like captions or empty space in the layout.
Captions: (caption)
Headers: (header)
Footers: (footer)
Page Numbers: (page_number)
Bounding Box Generation: For every element you identify, you must provide its precise location and dimensions on the page.
Coordinate System: The origin (0, 0) is the top-left corner of the page.
Units: Specify the units for the bounding box coordinates (e.g., pixels, points). Assume a standard page dimension if not specified (e.g., 8.5x11 inches at 96 DPI).
Format: The bounding box must be represented as [x_min, y_min, x_max, y_max].
Structured Output Format: Your final output must be a single JSON object. For each page, provide a list of "elements," where each element is an object containing its type, its text content (if applicable), and its bounding box.
Highest Accuracy and No Hallucinations:
Do not invent elements. Your analysis must be based on the logical and structural flow of the provided text.
Do not guess bounding boxes. The coordinates must be geometrically accurate. If the exact position of an element cannot be determined, do not create an entry for it.
Ensure that bounding boxes for different elements do not improperly overlap (e.g., a header's box should not contain a main paragraph's box).
Example of Desired Output:
Based on the example input above, the ideal output would be:
code
JSON
{
  "document_layout": [
    {
      "page_number": 1,
      "dimensions": {
        "width": 816,
        "height": 1056,
        "unit": "pixels"
      },
      "elements": [
        {
          "type": "heading_1",
          "content": "Document Title",
          "bounding_box": [100, 80, 716, 120]
        },
        {
          "type": "paragraph",
          "content": "This is the first paragraph of the document. It contains several sentences to illustrate a block of text.",
          "bounding_box": [100, 150, 716, 220]
        },
        {
          "type": "heading_2",
          "content": "Another Paragraph",
          "bounding_box": [100, 250, 716, 280]
        },
        {
          "type": "paragraph",
          "content": "This is the second paragraph. It follows the first one after a clear separation.",
          "bounding_box": [100, 290, 716, 340]
        }
      ]
    }
  ]
}"""

In [26]:

bounding_box_system_instructions = """
    Return bounding boxes as a JSON array with labels. Never return masks or code fencing. Limit to 25 objects.
    If an object is present multiple times, name them according to their unique characteristic (colors, size, position, unique characteristics, etc..).
"""
     

In [28]:
response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents=[file,data,sys_prompt_2],
    config=types.GenerateContentConfig(
        system_instruction=bounding_box_system_instructions,
        temperature=0
    )
)

In [29]:
new_data=parse_json(response.text)

In [ ]:

parsed_data = json.loads(new_data)

with open("output.json", "w", encoding="utf-8") as dump_file:
    json.dump(parsed_data, dump_file, ensure_ascii=False, indent=2)

In [10]:
import os
import json
import re
import tempfile
import fitz
import concurrent.futures
from google import genai
from google.genai import types
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Gemini client
client = genai.Client(api_key=os.getenv("gemini_api"))


def parse_json(json_output: str):
    """
    Extracts JSON content from Gemini response by removing ```json fences.
    """
    lines = json_output.splitlines()
    for i, line in enumerate(lines):
        if line.strip() == "```json":
            json_output = "\n".join(lines[i + 1:])
            json_output = json_output.split("```")[0]
            break
    return json_output


def split_pdf_to_pages(pdf_path: str) -> dict[int, str]:
    """
    Splits a PDF into individual one-page PDFs.

    Args:
        pdf_path (str): Path to the input PDF file.

    Returns:
        dict[int, str]: A mapping of page numbers to temporary single-page PDF paths.
    """
    output_dir = tempfile.mkdtemp(prefix="pdf_pages_")
    doc = fitz.open(pdf_path)
    page_dict = {}

    for page_num in range(len(doc)):
        single_page_doc = fitz.open()
        single_page_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
        output_path = os.path.join(output_dir, f"page_{page_num + 1}.pdf")
        single_page_doc.save(output_path)
        single_page_doc.close()
        page_dict[page_num + 1] = output_path

    doc.close()
    return page_dict


def ocr_single_pdf(pdf_path: str, ocr_prompt: str, output_dir: str):
    """
    Performs OCR on a single PDF using Gemini and saves JSON + Markdown outputs.

    Args:
        pdf_path (str): Path to the PDF file.
        ocr_prompt (str): OCR instruction prompt.
        output_dir (str): Directory to save results.

    Returns:
        tuple[str, str]: Paths to JSON and Markdown outputs.
    """
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    pdf_output_dir = os.path.join(output_dir, pdf_name)
    os.makedirs(pdf_output_dir, exist_ok=True)

    page_dict = split_pdf_to_pages(pdf_path)
    all_page_data = []
    markdown_output = []

    for page_number, page_path in page_dict.items():
        print(f"[{pdf_name}] Processing page {page_number}...")
        file = client.files.upload(file=page_path)
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[file],
            config=types.GenerateContentConfig(
                system_instruction=ocr_prompt,
                temperature=0
            )
        )

        data = {"page_number": page_number, "ocr": None}
        parsed_data = parse_json(response.text)

        if parsed_data:
            try:
                if isinstance(parsed_data, dict):
                    json_data = parsed_data
                else:
                    safe_str = re.sub(r'[\x00-\x1F\x7F]', ' ', parsed_data)
                    json_data = json.loads(safe_str)

                content = json_data.get("page", [{}])[0].get("content", "")
                data["ocr"] = content
                markdown_output.append(f"## Page {page_number}\n\n{content}\n")

            except Exception as e:
                print(f"[{pdf_name}] Error parsing page {page_number}: {e}")

        all_page_data.append(data)

    json_path = os.path.join(pdf_output_dir, f"{pdf_name}_ocr.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(all_page_data, f, indent=4, ensure_ascii=False)

    md_path = os.path.join(pdf_output_dir, f"{pdf_name}_ocr.md")
    with open(md_path, "w", encoding="utf-8") as f:
        f.write("# OCR Compiled Output\n\n" + "\n".join(markdown_output))

    print(f"[{pdf_name}] Completed. JSON: {json_path}, Markdown: {md_path}")
    return json_path, md_path


def ocr_folder_multithread(folder_path: str, output_dir: str = "ocr_results", max_threads: int = 4):
    """
    Processes all PDFs in a folder concurrently using multiple threads.

    Args:
        folder_path (str): Path to folder containing PDFs.
        ocr_prompt (str): Instruction prompt for Gemini OCR.
        output_dir (str): Directory where all outputs will be stored.
        max_threads (int): Number of concurrent OCR threads.

    Returns:
        list[tuple[str, str]]: List of tuples containing JSON and Markdown output paths for each PDF.
    """
    os.makedirs(output_dir, exist_ok=True)
    pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]

    if not pdf_files:
        print("No PDF files found in the given folder.")
        return []

    print(f"Found {len(pdf_files)} PDFs. Starting OCR with {max_threads} threads...\n")

    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {
            executor.submit(ocr_single_pdf, pdf, ocr_prompt, output_dir): pdf
            for pdf in pdf_files
        }

        for future in concurrent.futures.as_completed(futures):
            pdf_name = os.path.basename(futures[future])
            try:
                result = future.result()
                results.append(result)
                print(f"[{pdf_name}] OCR completed successfully.")
            except Exception as e:
                print(f"[{pdf_name}] Error during OCR: {e}")

    print("\nAll OCR tasks completed.")
    return results




In [12]:

folder_path = r"C:\Users\surya\OneDrive\Desktop\Projects\DocumentAI\Test_docs"
ocr_folder_multithread(folder_path, max_threads=4)


Found 3 PDFs. Starting OCR with 4 threads...

[brochure] Processing page 1...
[brochure_2] Processing page 1...
[OCCUPATION CERTIFICATE] Processing page 1...
[brochure_2] Processing page 2...
[brochure] Processing page 2...
[OCCUPATION CERTIFICATE] Processing page 2...
[brochure_2] Processing page 3...
[brochure] Processing page 3...
[OCCUPATION CERTIFICATE] Completed. JSON: ocr_results\OCCUPATION CERTIFICATE\OCCUPATION CERTIFICATE_ocr.json, Markdown: ocr_results\OCCUPATION CERTIFICATE\OCCUPATION CERTIFICATE_ocr.md
[OCCUPATION CERTIFICATE.pdf] OCR completed successfully.
[brochure] Processing page 4...
[brochure_2] Processing page 4...
[brochure] Processing page 5...
[brochure_2] Processing page 5...
[brochure] Processing page 6...
[brochure] Processing page 7...
[brochure] Processing page 8...
[brochure_2] Processing page 6...
[brochure] Processing page 9...
[brochure_2] Processing page 7...
[brochure] Processing page 10...
[brochure_2] Processing page 8...
[brochure] Processing page 

[('ocr_results\\OCCUPATION CERTIFICATE\\OCCUPATION CERTIFICATE_ocr.json',
  'ocr_results\\OCCUPATION CERTIFICATE\\OCCUPATION CERTIFICATE_ocr.md'),
 ('ocr_results\\brochure\\brochure_ocr.json',
  'ocr_results\\brochure\\brochure_ocr.md'),
 ('ocr_results\\brochure_2\\brochure_2_ocr.json',
  'ocr_results\\brochure_2\\brochure_2_ocr.md')]